In [ ]:
import ot
import sys
import tarfile

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset
import torchvision.models as models
from torchvision.datasets.utils import download_url

import numpy as np
from math import dist
#from lapsolver import solve_dense

#import pandas as pd

from scipy.stats import norm 
from scipy.stats import ncx2
from scipy.spatial.distance import cdist
from scipy.spatial.distance import pdist
from scipy.spatial.distance import euclidean
from scipy.spatial.distance import chebyshev
from scipy.optimize import linear_sum_assignment

import sys, os
sys.path.append("./Genealized barycenter problems") 
import pickle
from dataset.mnist import load_mnist

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

torch.manual_seed(0)

In [ ]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784', version=1)
mnist.keys()
dict_keys = ['data', 'target', 'frame' 'categories', 'feature_names', 'target_names', 'DESCR', 'details', 'url']


X, y = mnist['data'], mnist['target']
X = np.array(X)


"""Choose classes: 1, 4, 6, 9"""


X_1 = X[y=='1']
X_4 = X[y=='4']
X_6 = X[y=='6']
X_9 = X[y=='9']

print(len(X_1), len(X_4), len(X_6), len(X_9))

In [ ]:
"""
random module
classes : 1, 4, 6, 9
Pick 'bsize'-many data points from each of classes 1,4,6,9 randomly
n = bsize + 1 due to the ghost element
"""

bsize = 10
max_len = np.min([len(X_1), len(X_4), len(X_6), len(X_9)])

import random
random.seed(a=1, version=2)
random_ = [random.randint(0,max_len) for i in range(bsize)]  
random_ = np.array(random_)

np.random.seed(1)
x_1 = X_1[random_]
x_2 = X_4[random_]
x_3 = X_6[random_]
x_4 = X_9[random_]


"""
check the largest pairwise distance : range of adversarial budget, epsilon
Maximum infinity norm = 255.0
Maximum Eucldiean norm = 3418.7
the ratio = 13.40

"""



dist15 = cdist(x_1,x_2,'chebyshev')
dist16 = cdist(x_1,x_3,'chebyshev')
dist17 = cdist(x_1,x_4,'chebyshev')
dist56 = cdist(x_2,x_3,'chebyshev')
dist57 = cdist(x_2,x_4,'chebyshev')
dist67 = cdist(x_3,x_4,'chebyshev')

dist15_euc = cdist(x_1,x_2,'euclidean') 
dist16_euc = cdist(x_1,x_3,'euclidean') 
dist17_euc = cdist(x_1,x_4,'euclidean') 
dist56_euc = cdist(x_2,x_3,'euclidean') 
dist57_euc = cdist(x_2,x_4,'euclidean') 
dist67_euc = cdist(x_3,x_4,'euclidean')


max_dist = np.max([np.max(dist15), np.max(dist16), np.max(dist17), np.max(dist56), np.max(dist67), np.max(dist67)])
max_dist_euc = np.max([np.max(dist15_euc), np.max(dist16_euc), np.max(dist17_euc), np.max(dist56_euc), np.max(dist67_euc), np.max(dist67_euc)])
infty_vs_euc = max_dist_euc / max_dist

print(max_dist, max_dist_euc, infty_vs_euc)

In [ ]:
from colored_rips import basic_rips, colored_rips
from entropic import approximate_MOT
from indicator_solver import solve

In [ ]:
m = 4    
n = 10
data = [x_1, x_2, x_3, x_4,]
eta = 0.3  
eps_range = np.arange(120, 220, 10) 

entropic_cost_list = []
exact_cost_list = []
for epsilon in eps_range:
    r_ks = np.ones((m,n+1)) / (n) 
    X, C = approximate_MOT(data, r_ks, epsilon, chebyshev, eta)
    entropic_cost_list.append((X*C).sum())

    groupings = colored_rips(data, epsilon, m, distance=chebyshev)
    op, cost = solve(groupings, data, [r for r in np.ones((m,n)) / n])
    exact_cost_list.append(cost)
    
    
np.savetxt('entropic_cost_MNIST.csv', entropic_cost_list, delimiter=',')
np.savetxt('exact_cost_MNIST.csv', exact_cost_list, delimiter=',')
np.savetxt('eps_range_MNIST.csv', eps_range, delimiter=',')

In [ ]:
"""
m = number of classes
n = bsize = 20
data = class 1, 4, 6, 9
eta = entropic parameter
eps range = range of adversarial budget, epsilon

"""





m = 4
n = 10
data = [x_1, x_2, x_3, x_4,]
eta = 0.3  
euc_eps_range = np.arange(700, 1700, 100) 

euc_entropic_cost_list = []
euc_exact_cost_list = []
for epsilon in euc_eps_range:
    r_ks = np.ones((m,n+1)) / (n) 
    X, C = approximate_MOT(data, r_ks, epsilon, euclidean, eta)
    euc_entropic_cost_list.append((X*C).sum())

    groupings = colored_rips(data, epsilon, m, distance=euclidean)
    op, cost = solve(groupings, data, [r for r in np.ones((m,n)) / n])
    euc_exact_cost_list.append(cost)
    
    
np.savetxt('euc_entropic_cost_MNIST.csv', euc_entropic_cost_list, delimiter=',')
np.savetxt('euc_exact_cost_MNIST.csv', euc_exact_cost_list, delimiter=',')
np.savetxt('euc_eps_range_MNIST.csv', euc_eps_range, delimiter=',')